Control Flow Prediction
===

Process Prediction that focuses only on the control flow perspective of processes

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from exp.eventlog import *

In [3]:
from exp.dl_utils import *

In [4]:
import editdistance as ed

# Import Data

In [5]:
log=import_xes(untar_data(URLs.BPIC_2012)).events

In [6]:
log

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00
5,173688,5,10862,COMPLETE,A_ACCEPTED,2011-10-01 09:42:43.308000+00:00
6,173688,6,10862,COMPLETE,O_SELECTED,2011-10-01 09:45:09.243000+00:00
7,173688,7,10862,COMPLETE,A_FINALIZED,2011-10-01 09:45:09.243000+00:00
8,173688,8,10862,COMPLETE,O_CREATED,2011-10-01 09:45:11.197000+00:00
9,173688,9,10862,COMPLETE,O_SENT,2011-10-01 09:45:11.380000+00:00


# Control Flow List

In [7]:
#export
class ListContainer():
    def __init__(self, items): self.items = listify(items)
    def __getitem__(self, idx):
        if isinstance(idx, (int,slice)): return self.items[idx]
        if isinstance(idx[0],bool):
            assert len(idx)==len(self) # bool mask
            return [o for m,o in zip(idx,self.items) if m]
        return [self.items[i] for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res

In [8]:
class ControlFlowList(ListContainer):

    def __init__(self,items,path='.'): 
        super().__init__(items)
        self.path=Path(path)
        
    @classmethod
    def from_df(cls,df,col='concept:name',trace_id='trace_id'):
        items=[]
        for n,g in df.groupby('trace_id'):
            items.append(list(g[col]))
        
        return cls(items)
    def new(self, items, cls=None):
        if cls is None: cls=self.__class__
        return cls(items, self.path)
    
    def get(self,o): return "->".join([str(x) for x in listify(self.items[o])])

    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{[self.get(i) for i in range(2)]}'
        if len(self)>2: res = res[:-1]+ '...]'
        return res
    




In [9]:
cfl=ControlFlowList.from_df(log)
cfl

ControlFlowList (13087 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->A_FINALIZED->O_SELECTED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SELECTED->O_CANCELLED->O_CREATED->O_SENT->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_

In [10]:
cfl[0]

['A_SUBMITTED',
 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'A_ACCEPTED',
 'O_SELECTED',
 'A_FINALIZED',
 'O_CREATED',
 'O_SENT',
 'W_Nabellen offertes',
 'W_Completeren aanvraag',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'O_SENT_BACK',
 'W_Valideren aanvraag',
 'W_Nabellen offertes',
 'W_Valideren aanvraag',
 'A_REGISTERED',
 'A_APPROVED',
 'O_ACCEPTED',
 'A_ACTIVATED',
 'W_Valideren aanvraag']

# Splitting Data in Train, Test and Valid

In [11]:
import random

In [12]:
#export

def random_splitter(fn, p_valid): return random.random() < p_valid
def split_by_func(items, f):
    mask = [f(o) for o in items]
    # `None` values will be filtered out
    f = [o for o,m in zip(items,mask) if m==False]
    t = [o for o,m in zip(items,mask) if m==True ]
    return f,t

class SplitData():
    def __init__(self, train, valid, test): self.train,self.valid,self.test = train,valid,test
    
    @classmethod
    def split_by_func(cls, il, f, Test=True):
        train, valid = map(il.new, split_by_func(il.items, f))
        test=None
        if Test:
            train, test = map(train.new, split_by_func(train.items, f))

        return cls(train,valid,test)

    def __repr__(self): return f'{self.__class__.__name__}\nTrain: {self.train}\nValid: {self.valid}\nTest: {self.test}\n'

In [13]:
sd=SplitData.split_by_func(cfl, partial(random_splitter, p_valid=0.1));sd

SplitData
Train: ControlFlowList (10536 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->A_FINALIZED->O_SELECTED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SELECTED->O_CANCELLED->O_CREATED->O_SENT->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabe

In [14]:
sd.train

ControlFlowList (10536 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->A_FINALIZED->O_SELECTED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SELECTED->O_CANCELLED->O_CREATED->O_SENT->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_

# Encoding

## Tokenization

In [15]:
UNK, PAD, BOT, EOT = "xxunk xxpad xxbot xxeot ".split()

def add_eot_bot(x): return [BOT] + x + [EOT]
default_spec_tok = [UNK, PAD, BOT, EOT]
default_pre_rules = []
default_post_rules = [add_eot_bot]

In [16]:
from concurrent.futures import ProcessPoolExecutor

def parallel(func, arr, max_workers=4):
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results
    

In [17]:
class TokenizeProcessor():
    def __init__(self, lang="en", chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize,self.max_workers = chunksize,max_workers        
        self.pre_rules  = default_pre_rules  if pre_rules  is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules

    def proc_chunk(self, args):
        i,chunk = args
        docs = [compose(t, self.pre_rules) for t in chunk]
        docs = [compose(t, self.post_rules) for t in docs]
        return docs

    def __call__(self, items): 
        toks = []
        if isinstance(items[0], Path): items = [read_file(i) for i in items]
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])
    
    def proc1(self, item): return self.proc_chunk([item])[0]
    

In [18]:
tp = TokenizeProcessor()

In [19]:
cfl.get(0)

'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag'

In [20]:
' • '.join(tp(cfl[:2])[0])

'xxbot • A_SUBMITTED • A_PARTLYSUBMITTED • A_PREACCEPTED • W_Completeren aanvraag • W_Completeren aanvraag • A_ACCEPTED • O_SELECTED • A_FINALIZED • O_CREATED • O_SENT • W_Nabellen offertes • W_Completeren aanvraag • W_Nabellen offertes • W_Nabellen offertes • W_Nabellen offertes • W_Nabellen offertes • W_Nabellen offertes • O_SENT_BACK • W_Valideren aanvraag • W_Nabellen offertes • W_Valideren aanvraag • A_REGISTERED • A_APPROVED • O_ACCEPTED • A_ACTIVATED • W_Valideren aanvraag • xxeot'

## Numericalization

In [21]:
#export
import collections

class NumericalizeProcessor():
    def __init__(self, vocab=None, max_vocab=60000, min_freq=0,spec_token=None,unk_token=None): 
        self.vocab,self.max_vocab,self.min_freq = vocab,max_vocab,min_freq
        self.spec_token = default_spec_tok if spec_token is None else spec_token
        self.unk_token = UNK if unk_token is None else unk_token

    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            freq = Counter(p for o in items for p in o)
            self.vocab = [o for o,c in freq.most_common(self.max_vocab) if c >= self.min_freq]
            for o in reversed(default_spec_tok):
                if o in self.vocab: self.vocab.remove(o)
                self.vocab.insert(0, o)
        if getattr(self, 'otoi', None) is None:
            self.otoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.vocab)}) 
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi.get(o, self.unk_token) for o in item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

In [22]:
proc_tok,proc_num = TokenizeProcessor(),NumericalizeProcessor()

In [23]:
proc_num.vocab

## Application of Processors and Labeling

In [24]:
#export

def _label_by_func(ds, f, cls=ControlFlowList): return cls([f(o) for o in ds.items], path=ds.path)

#This is a slightly different from what was seen during the lesson,
#   we'll discuss the changes in lesson 11
class LabeledData():
    def process(self, il, proc): return il.new(compose(il.items, proc))

    def __init__(self, x, y, proc_x=None, proc_y=None):
        self.x,self.y = self.process(x, proc_x),self.process(y, proc_y)
        self.proc_x,self.proc_y = proc_x,proc_y
        
    def __repr__(self): return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx): return self.x[idx],self.y[idx]
    def __len__(self): return len(self.x)
    
    def x_obj(self, idx): return self.obj(self.x, idx, self.proc_x)
    def y_obj(self, idx): return self.obj(self.y, idx, self.proc_y)
    
    def obj(self, items, idx, procs):
        isint = isinstance(idx, int) or (isinstance(idx,torch.LongTensor) and not idx.ndim)
        item = items[idx]
        for proc in reversed(listify(procs)):
            item = proc.deproc1(item) if isint else proc.deprocess(item)
        return item

    @classmethod
    def label_by_func(cls, il, f, proc_x=None, proc_y=None):
        return cls(il, _label_by_func(il, f), proc_x=proc_x, proc_y=proc_y)

def label_by_func(sd, f, proc_x=None, proc_y=None):
    train = LabeledData.label_by_func(sd.train, f, proc_x=proc_x, proc_y=proc_y)
    valid = LabeledData.label_by_func(sd.valid, f, proc_x=proc_x, proc_y=proc_y)
    test =None if sd.test is None else LabeledData.label_by_func(sd.test, f, proc_x=proc_x, proc_y=proc_y)

    return SplitData(train,valid,test)

In [25]:
%time ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])

CPU times: user 262 ms, sys: 485 ms, total: 747 ms
Wall time: 892 ms


In [26]:
ll

SplitData
Train: LabeledData
x: ControlFlowList (10536 items)
['2->9->10->12->4->4->16->13->17->14->15->5->4->5->5->5->5->5->19->7->5->7->21->22->24->23->7->3', '2->9->10->12->4->4->4->4->16->17->13->14->15->5->4->5->5->5->13->18->14->15->5->5->5->5->5->19->7->5->7->7->7->7->7->24->22->21->23->7->3'...]
y: ControlFlowList (10536 items)
['0', '0'...]

Valid: LabeledData
x: ControlFlowList (1356 items)
['2->9->10->12->4->4->4->4->4->4->4->4->16->13->17->14->15->5->4->5->5->5->5->5->5->5->5->5->5->5->5->5->5->5->19->7->5->7->6->7->6->6->6->6->6->6->6->7->6->7->6->7->6->6->6->7->6->7->6->7->6->6->6->6->6->7->6->7->7->7->7->7->24->23->21->22->7->3', '2->9->10->8->8->12->4->8->4->4->4->4->4->4->4->4->4->16->17->13->14->15->5->4->5->5->5->18->13->14->15->5->5->5->5->5->11->25->5->3'...]
y: ControlFlowList (1356 items)
['0', '0'...]

Test: LabeledData
x: ControlFlowList (1195 items)
['2->9->10->12->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->20->4->3', '2->9->10->12

# Language Model DataLoader and DataBunch

In [27]:
#export
class CF_LM_Dataset():
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data,self.bs,self.bptt,self.shuffle = data,bs,bptt,shuffle
        total_len = sum([len(t) for t in data.x])
        self.n_batch = total_len // bs
        self.batchify()
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def batchify(self):
        texts = self.data.x
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([tensor(t) for t in texts])
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

In [28]:
#export
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(CF_LM_Dataset(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(CF_LM_Dataset(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs))

def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

In [29]:
dl = DataLoader(CF_LM_Dataset(ll.valid, shuffle=True), batch_size=64)

In [30]:
iter_dl = iter(dl)
xb,yb = next(iter_dl)


In [31]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

# AWD_LSTM

In [32]:
#export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [33]:
x = torch.randn(10,10)
mask = dropout_mask(x, (10,10), 0.5); mask

tensor([[0., 0., 0., 2., 2., 2., 2., 2., 0., 2.],
        [2., 2., 0., 2., 2., 0., 2., 0., 2., 0.],
        [0., 0., 0., 0., 0., 2., 2., 2., 0., 0.],
        [0., 0., 2., 0., 2., 2., 2., 2., 0., 0.],
        [0., 2., 2., 2., 2., 0., 0., 2., 0., 2.],
        [2., 0., 0., 2., 2., 2., 2., 0., 0., 0.],
        [0., 0., 2., 2., 0., 2., 0., 2., 2., 2.],
        [2., 2., 0., 2., 0., 2., 2., 0., 0., 2.],
        [2., 0., 0., 2., 0., 2., 2., 0., 2., 2.],
        [0., 0., 0., 0., 2., 2., 2., 0., 2., 0.]])

In [34]:
(x*mask).std(),x.std()

(tensor(1.9047), tensor(1.1420))

In [35]:
#export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        return x * m

In [36]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input.shape, dp(tst_input).shape

(torch.Size([3, 3, 7]), torch.Size([3, 3, 7]))

In [37]:
#export
import warnings

WEIGHT_HH = 'weight_hh_l0'

class WeightDropout(nn.Module):
    def __init__(self, module, weight_p=[0.], layer_names=[WEIGHT_HH]):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            self.module._parameters[layer] = F.dropout(w, p=self.weight_p, training=False)

    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)

    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

In [38]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, WEIGHT_HH)

Parameter containing:
tensor([[-0.4933, -0.5138],
        [-0.4675,  0.1721],
        [ 0.0703,  0.0150],
        [-0.5622,  0.1879],
        [-0.2864,  0.4550],
        [ 0.5857,  0.1038],
        [ 0.0974, -0.6903],
        [ 0.6597, -0.1791]], requires_grad=True)

In [39]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, WEIGHT_HH)

tensor([[-0.0000, -0.8564],
        [-0.0000,  0.0000],
        [ 0.0000,  0.0250],
        [-0.0000,  0.3132],
        [-0.4773,  0.7584],
        [ 0.9762,  0.0000],
        [ 0.0000, -0.0000],
        [ 1.0994, -0.0000]], grad_fn=<MulBackward0>)

In [40]:
#export
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb,self.embed_p = emb,embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [41]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[-0.0000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
        [-0.9490, -0.7544, -0.4486, -2.1250, -4.5645, -0.8189,  3.0518],
        [-0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

In [42]:
#export
def to_detach(h):
    "Detaches `h` from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [43]:
#export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        super().__init__()
        self.bs,self.emb_sz,self.n_hid,self.n_layers = 1,emb_sz,n_hid,n_layers
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz), 1,
                             batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        bs,sl = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        raw_output = self.input_dp(self.emb_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output) 
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

In [44]:
#export
class LinearDecoder(nn.Module):
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        self.output_dp = RNNDropout(output_p)
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight
        else: init.kaiming_uniform_(self.decoder.weight)

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1]).contiguous()
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [45]:
#export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [46]:
#export
def get_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, output_p=0.4, hidden_p=0.2, input_p=0.6, 
                       embed_p=0.1, weight_p=0.5, tie_weights=True, bias=True):
    rnn_enc = AWD_LSTM(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                       hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.emb if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [47]:
tok_pad = proc_num.vocab.index(PAD)

In [48]:
tst_model = get_language_model(len(proc_num.vocab), 300, 300, 2, tok_pad)


In [49]:
z=tst_model(xb)
z

(tensor([[-0.0089, -0.0296,  0.0149,  ...,  0.0038, -0.0200, -0.0265],
         [-0.0084, -0.0466,  0.0240,  ...,  0.0016, -0.0351, -0.0431],
         [-0.0046, -0.0643,  0.0267,  ..., -0.0004, -0.0484, -0.0549],
         ...,
         [ 0.0281, -0.0688,  0.0128,  ...,  0.0408, -0.0524, -0.0482],
         [ 0.0280, -0.0705,  0.0110,  ...,  0.0440, -0.0496, -0.0485],
         [ 0.0298, -0.0720,  0.0104,  ...,  0.0462, -0.0485, -0.0507]],
        grad_fn=<AddmmBackward>),
 [tensor([[[ 0.0078, -0.0128,  0.0082,  ...,  0.0259,  0.0089, -0.0087],
           [ 0.0159, -0.0148, -0.0070,  ...,  0.0275, -0.0200, -0.0211],
           [ 0.0227,  0.0037,  0.0066,  ...,  0.0187, -0.0127,  0.0027],
           ...,
           [ 0.0136,  0.0458, -0.0086,  ...,  0.0201,  0.0287, -0.0668],
           [-0.0037,  0.0285, -0.0088,  ...,  0.0175,  0.0222, -0.0626],
           [-0.0160,  0.0191, -0.0154,  ...,  0.0283,  0.0271, -0.0338]],
  
          [[ 0.0012,  0.0061, -0.0123,  ..., -0.0080, -0.0023,  0.0

In [50]:
# export
def cross_entropy_flat(input, target):
    bs,sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

def accuracy_flat(input, target):
    bs,sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [51]:
decoded, raw_outputs, outputs = z
accuracy_flat(decoded,yb)


tensor(0.0453)

In [52]:
cross_entropy_flat(decoded,yb)

tensor(3.3209, grad_fn=<NllLossBackward>)

# Training Loop with AWD-LSTM

In [53]:
#export
class GradientClipping(Callback):
    def __init__(self, clip=None): self.clip = clip
    def after_backward(self):
        if self.clip:  nn.utils.clip_grad_norm_(self.run.model.parameters(), self.clip)

In [54]:
#export
class RNNTrainer(Callback):
    def __init__(self, α, β): self.α,self.β = α,β
    
    def after_pred(self):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = self.pred[1],self.pred[2]
        self.run.pred = self.pred[0]
    
    def after_loss(self):
        #AR and TAR
        if self.α != 0.:  self.run.loss += self.α * self.out[-1].float().pow(2).mean()
        if self.β != 0.:
            h = self.raw_out[-1]
            if len(h)>1: self.run.loss += self.β * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
                
    def begin_epoch(self):
        pass
        #Shuffle the texts at the beginning of the epoch
        #if hasattr(self.dl.dataset, "batchify"): self.dl.dataset.batchify()

In [55]:
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [56]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(proc_num.vocab), emb_sz, nh, nl, tok_pad, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)

In [57]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressBarCallback]

In [58]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [59]:
learn.fit(20)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,1.649115,0.526422,0.823463,0.736719,00:01
1,0.696616,0.778191,0.511666,0.831473,00:01
2,0.557296,0.818050,0.463992,0.846503,00:00
3,0.485490,0.834515,0.446155,0.850037,00:00
4,0.453156,0.842708,0.443367,0.846801,00:00
5,0.425081,0.847418,0.426659,0.854799,00:00
6,0.416509,0.846516,0.422325,0.856176,00:00
7,0.398458,0.851313,0.419815,0.854613,00:00
8,0.404604,0.850446,0.426260,0.853348,00:00
9,0.390960,0.851514,0.416056,0.854650,00:00


# Testing

In [60]:
sd.test

ControlFlowList (1195 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_CANCELLED->W_Completeren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_DECLINED->W_Completeren aanvraag'...]

## Next Step Prediction

In [61]:
def process_data_for_next_step_prediction(test,startIndex=1):
    xs,ys=[],[]
    for trace in progress_bar(test.items):
        for i in range(startIndex,len(listify(trace))):
            x,y=[],[]
            xs.append(tensor([trace[:i]]))
            ys.append(tensor([[trace[i]]]))
    return xs,tensor(ys)
x,y=process_data_for_next_step_prediction(ll.test.x)

In [62]:
learn.model.cpu()
learn.model.eval()
preds=[]
for e in progress_bar(x):
    learn.model.reset()
    pred=learn.model(e)
    preds.append(pred[0][-1])
pred_stack=torch.stack(preds)
accuracy(pred_stack,(y))

tensor(0.8593)

In [63]:
%time pred_stack=torch.stack(preds)

CPU times: user 55.8 ms, sys: 0 ns, total: 55.8 ms
Wall time: 54.8 ms


In [65]:
%time accuracy(pred_stack,(y))

CPU times: user 1.48 ms, sys: 24 µs, total: 1.5 ms
Wall time: 801 µs


tensor(0.8593)

## Suffix Prediction

Since the model is a LM we need to sample data to get the final suffix prediction

In [66]:
def process_data_for_suffix_prediction(test,startIndex=1):
    xs,ys=[],[]
    for trace in progress_bar(test.items):
        for i in range(startIndex,len(listify(trace))):
            x,y=[],[]
            xs.append(tensor([trace[:i]]))
            ys.append(tensor([trace[i:]]))
    return xs,ys
xs,ys=process_data_for_suffix_prediction(ll.test.x)

In [83]:
maxl=-1
for x in xs:
    if len(x[0])>maxl: maxl=len(x[0])
maxl

171

In [ ]:
maxl

In [77]:
len(xs[0])

1

In [ ]:
def predict_suffix(model,xs,ys):
    rl=[]
    model.cpu()
    model.eval()
    for x,y in progress_bar(zip(xs,ys)):
        model.reset()
        t=torch.stack([tensor(x[c]).float() for c in range(len(list(df)))])
        p=tensor(-1)
        res=[]
        while p.int()!=3: # 3: eos token
            pred=model(t[None])
            pred=pred[0][-1]
            p=torch.multinomial(torch.softmax(pred,0),1).float()
           # p=torch.argmax(pred,0).float()[None]
            if p.int()!=3 or len(res)==0: res.append(p)
            k=torch.cat((t[3],p))
            t=torch.stack([k for c in range(len(list(df)))])


        res=torch.cat(res,0).int().tolist()
        rl.append(res)
    return rl

In [ ]:
x=xs[0]
y=ys[0]

In [ ]:
len(xs)

In [84]:
preds=[]
for x in progress_bar(xs):
    learn.model.reset()
    res=[]
    t=x
    p=tensor(-1)
    k=0
    while p.int()!=3 and k<100:
        k+=1
        pred=learn.model(t)
        pred=pred[0][-1]
        p=torch.multinomial(torch.softmax(pred,0),1)[None]
        res.append(p)
        t=torch.cat((t,p),dim=1)
    preds.append(tensor(res).tolist())

In [ ]:
preds[0:10]

In [ ]:
y

In [ ]:
preds2=[]
for x in progress_bar(xs):
    learn.model.reset()
    res=[]
    t=x
    p=tensor(-1)
    k=0
    while p.int()!=3:
        k+=1
        pred=learn.model(t)
        pred=pred[0][-1]
        p=torch.multinomial(torch.softmax(pred,0),1)[None]
        res.append(p)
        t=torch.cat((t,p),dim=1)
    preds2.append(tensor(res).tolist())